In [25]:
import os
import tkinter as tk
from io import BytesIO
from tkinter import filedialog
from tkinter import messagebox
from tkinter import ttk
from PIL import Image
import PIL
from docx import Document
from docx.shared import Mm
from datetime import datetime

In [31]:
# Function to crop an image to square
def crop_to_square(image_file:BytesIO) -> Image.Image:
        img = Image.open(image_file)

        if img.width != img.height:
            min_dimension = min(img.width, img.height)
            left = (img.width - min_dimension) / 2
            top = (img.height - min_dimension) / 2
            right = (img.width + min_dimension) / 2
            bottom = (img.height + min_dimension) / 2
            img = img.crop((left, top, right, bottom))
        
        img = img.convert("RGB")

        return img


In [55]:
# Function to check if an image exceeds the size limit
def is_image_size_exceed_limit(image_file:str) -> bool:
    try:
        img = Image.open(image_file)
        return True
    except Exception as e:
        return False

# Function to confirm with the user whether to continue with images exceeding the size limit
def confirm_continue_with_large_images(exceeding_images: list) -> bool:
    if not exceeding_images:
        return True  # No images exceeded the size limit, continue with the process

    # Generate a message with the list of exceeding images
    message = f"Found images exceed the size limit and will be excluded.\n\n"
    message += "\n\nDo you want to continue?"

    response = messagebox.askquestion("Image Size Warning", message)
    return response.lower() == "yes"

In [66]:
def create_word_document(output_folder:str, selected_folder:str, repetition_line:int) -> None:
    doc = Document()
    exceeding_images = []
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    

    image_files = [os.path.join(selected_folder, filename) for filename in os.listdir(selected_folder) if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.gif'))]

    if not image_files:
        messagebox.showwarning("Warning", f"No image files found in the selected folder!")
        return

    
    # Set page margins (0.5 inch for all borders)
    section = doc.sections[0]
    section.left_margin = Mm(12.7)  # 0.5 inch
    section.right_margin = Mm(12.7)  # 0.5 inch
    section.top_margin = Mm(12.7)  # 0.5 inch
    section.bottom_margin = Mm(12.7)  # 0.5 inch
    
    current_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    output_path = os.path.join(output_folder, f"{current_time}.docx")

    
    for image_file in image_files:
        
        is_exceed = is_image_size_exceed_limit(image_file)
        
        if not is_exceed and len(exceeding_images)!=0:
            exceeding_images.append(image_file)    
            continue
        
        if not is_exceed:         
            exceeding_images.append(image_file)   
            stop_sign = confirm_continue_with_large_images(exceeding_images)

            if not stop_sign:
                messagebox.showinfo("Info", "Operation canceled.")
                return
        
            continue
        
        # Crop the image to square and convert to RGB
        img = crop_to_square(image_file)
        
        if img is None:
            continue  # Skip this image and continue with the next one

        # Add the picture to the paragraph with repetitions
        img.save("temp_square_image.jpg")  # Save the square image temporarily
        
        # Repeat the image in the same line
        run = doc.add_paragraph().add_run()
        for _ in range(repetition_line):
            for i in range(11):
                run.add_picture("temp_square_image.jpg", width=Mm(16), height=Mm(16))
                if i<10: run.add_text(" ")
        
        
        os.remove("temp_square_image.jpg")  # Remove the temporary file
    
    
    if len(exceeding_images) == len(image_files):
        messagebox.showwarning("Warning", "All images are exceeding the maximum size. Please, choose another folder or resize your images!")
        
    doc.save(output_path)
    messagebox.showinfo("Success", f"Document saved to:\n{output_path}")

In [67]:
# Define a function to handle the Convert button click
def convert_button_click(selected_folder_var, output_folder_var, repetition_lines_var):
    selected_folder = selected_folder_var.get()
    output_folder = output_folder_var.get()
    repetition_lines_str = repetition_lines_var.get()

    if not selected_folder:
        messagebox.showwarning("Warning", f"Please, select the input folder!")
        return

    if not repetition_lines_str:
        messagebox.showwarning("Warning", "Please enter a value for Repetition Lines.")
        return

    try:
        repetition_lines = int(repetition_lines_str)  
    except ValueError:
        messagebox.showwarning("Warning", "Repetition Lines must be a valid integer.")
        return

    create_word_document(output_folder, selected_folder, repetition_lines)

# Function to select a folder and set the folder path in the corresponding Entry widget
def select_folder(variable):
    folder_path = filedialog.askdirectory()
    if folder_path:
        variable.set(folder_path)

In [68]:
# Function to list image files in a folder
def list_image_files(folder_path):
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']  # Add more extensions if needed
    image_files = []

    for filename in os.listdir(folder_path):
        if any(filename.lower().endswith(ext) for ext in image_extensions):
            image_files.append(os.path.join(folder_path, filename))

    return image_files

# Function to handle folder selection
def select_folder(folder_var, image_listbox):
    folder_path = filedialog.askdirectory()
    if folder_path:
        folder_var.set(folder_path)
        # List image files in the selected folder and update the image listbox
        image_files = list_image_files(folder_path)
        image_listbox.delete(0, tk.END)  # Clear existing entries
        for image_file in image_files:
            image_listbox.insert(tk.END, os.path.basename(image_file))

# Function to reset the listbox focus when an item is clicked
def reset_focus(event):
    event.widget.selection_clear(0, tk.END)
    event.widget.focus_set()

# Function to create the main application window
def create_app_window():
    app = tk.Tk()
    app.title("Image to Word Converter")

    # Set the minimum window size (width x height)
    app.minsize(390, 350)  # Adjust the values as needed

    # Variables to store folder paths
    selected_folder_var = tk.StringVar()
    output_folder_var = tk.StringVar(value=os.path.expanduser('~/Desktop'))  # Set the desktop folder as the default value

    # Create and configure widgets for selecting input folder
    selected_folder_label = ttk.Label(app, text="Select Input Folder:")
    selected_folder_entry = ttk.Entry(app, textvariable=selected_folder_var, state="readonly")
    select_folder_button = ttk.Button(app, text="Select Folder", command=lambda: select_folder(selected_folder_var, image_listbox))

    # Create and configure widgets for selecting output folder
    output_folder_label = ttk.Label(app, text="Select Output Folder:")
    output_folder_entry = ttk.Entry(app, textvariable=output_folder_var, state="readonly")
    select_output_folder_button = ttk.Button(app, text="Select Folder", command=lambda: select_folder(output_folder_var, image_listbox))

    # Create a listbox to display image files in the selected folder
    image_listbox = tk.Listbox(app, selectmode=tk.MULTIPLE)
    image_listbox.grid(row=2, column=0, columnspan=4, padx=5, pady=5, sticky="nsew")
    image_listbox.bind("<ButtonRelease-1>", reset_focus)  # Bind the reset_focus function to left mouse button release event

    # Create a scrollbar for the listbox
    scrollbar = ttk.Scrollbar(app, orient=tk.VERTICAL, command=image_listbox.yview)
    scrollbar.grid(row=2, column=4, sticky="ns")
    image_listbox.configure(yscrollcommand=scrollbar.set)

    # Grid layout for widgets with padding for margin
    selected_folder_label.grid(row=0, column=0, sticky="w", padx=10, pady=10)
    selected_folder_entry.grid(row=0, column=1, padx=5, pady=10, columnspan=2, sticky="ew")
    select_folder_button.grid(row=0, column=3, padx=10, pady=10)

    output_folder_label.grid(row=1, column=0, sticky="w", padx=10, pady=10)
    output_folder_entry.grid(row=1, column=1, padx=5, pady=10, columnspan=2, sticky="ew")
    select_output_folder_button.grid(row=1, column=3, padx=10, pady=10)

    # Add a label and entry for user input
    repetitions_label = ttk.Label(app, text="Repetition lines:")
    repetitions_label.grid(row=3, column=0, sticky="w", padx=10, pady=10)

    repetion_lines_var = tk.StringVar()
    repetitions_entry = ttk.Entry(app, textvariable=repetion_lines_var)
    repetitions_entry.grid(row=3, column=1, padx=5, pady=10, columnspan=2, sticky="ew")

    # Create the Convert button with the new function
    convert_button = ttk.Button(app, text="Convert", command=lambda: convert_button_click(selected_folder_var, output_folder_var, repetion_lines_var))
    convert_button.grid(row=4, column=0, columnspan=4, pady=10)

    app.mainloop()

# Call the function to create the main application window
create_app_window()

c:\Users\USER\Music\PROGRAMING\Projects\20231005-resize_app\venv\lib\site-packages\PIL\Image.py:3179: DecompressionBombWarning: Image size (149975001 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
c:\Users\USER\Music\PROGRAMING\Projects\20231005-resize_app\venv\lib\site-packages\PIL\Image.py:3179: DecompressionBombWarning: Image size (99980001 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
